# PyTorch qGAN Implementation

## Description
A PyTorch-based Quantum Generative Adversarial Network algorithm.

The qGAN [1] is a hybrid quantum-classical algorithm used for generative modeling tasks.

This adaptive algorithm uses the interplay of a generator given by a parameterized quantum
circuit and a discriminator given by a classical neural network to learn the probability
distribution underlying given training data.

The generator and discriminator are trained in alternating optimization steps, where the
generator aims at generating samples which the discriminator classifies as training data
samples and the discriminator tries to differentiate between training data samples
and data samples from the generator. The final goal is for the quantum generator to learn
a representation for the training data's underlying probability distribution.
The trained quantum generator can, thus, be used to load a quantum state
which is an approximate model of the target distribution.

**References:**

[1] Zoufal et al.,
    `Quantum Generative Adversarial Networks for learning and loading random distributions
    <https://www.nature.com/articles/s41534-019-0223-2>`_

In [3]:
# Necessary imports

import numpy as np
from scipy.stats import entropy
from typing import Union, List, Iterable
import matplotlib.pyplot as plt
%matplotlib inline

from torch import Tensor, stack, manual_seed
from torch.utils.data import DataLoader
from torch.autograd import Variable
from torch.optim import Adam
import torch.nn as nn
import torch.nn.functional as F


from qiskit import Aer, QuantumCircuit
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.opflow import Gradient, StateFn
from qiskit.circuit.library import TwoLocal
from qiskit.circuit import ParameterExpression, ParameterVector
from qiskit_machine_learning.neural_networks import CircuitQNN
from qiskit_machine_learning.connectors import TorchConnector
from qiskit_machine_learning.datasets.dataset_helper import discretize_and_truncate

# Set seed for random generators
algorithm_globals.random_seed = 42
np.random.seed = 42
manual_seed(42)

### Data and Representation

First, we need to load our training data $X$. In this tutorial, the training data
is given by samples from a 2D multivariate normal distribution.

The trained generator corresponds to an $n$-qubit quantum state
\begin{equation}
|g_{\text{trained}}\rangle=\sum\limits_{j=0}^{k-1}\sqrt{p_{j}}|x_{j}\rangle,
\end{equation}
where the basis states $|x_{j}\rangle$ represent the data items in the training data set
$X={x_0, \ldots, x_{k-1}}$ with $k\leq 2^n$ and $p_j$ refers to the sampling probability
of $|x_{j}\rangle$.

To facilitate this representation, we need to map the samples from the multivariate
normal distribution to discrete values. The number of values that can be represented
depends on the number of qubits used for the mapping.
Hence, the data resolution is defined by the number of qubits.
If we use $3$ qubits to represent one feature, we have 2^3=8 discrete values.

In [4]:
# Load the training data
training_data = np.random.default_rng().multivariate_normal(mean=[0., 0.], cov=[[1, 0], [0, 1]], size=1000, check_valid='warn',
                                                        tol=1e-8, method='svd')
# Define minimal and maximal values for the training data
bounds_min = np.percentile(training_data, 5, axis=0)
bounds_max = np.percentile(training_data, 95, axis=0)
bounds = []
for i, _ in enumerate(bounds_min):
    bounds.append([bounds_min[i], bounds_max[i]])

# Determine data resolution for each dimension of the training data in terms
# of the number of qubits used to represent each data dimension.
data_dim = [3, 3]

# Pre-processing, i.e., discretization of the data (gridding)
(training_data,
data_grid,
grid_elements,
prob_data ) = discretize_and_truncate(
training_data,
np.array(bounds),
data_dim,
return_data_grid_elements=True,
return_prob=True,
prob_non_zero=True,
)

# Define the training batch size
batch_size = 300
dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True, drop_last=True)

### Specify Backend
Next, we need to choose a backend that is used to run the quantum generator.
The presented method is compatible with all shot-based backends (qasm, fake hardware, real hardware) provided by Qiskit.

In [5]:
backend = Aer.get_backend('aer_simulator')
qi = QuantumInstance(backend, shots = batch_size)

adapted from [PyTorch GAN](https://github.com/eriklindernoren/PyTorch-GAN/blob/master/implementations/gan/gan.py)
TODO

### Definition of quantum generator and the respective gradient
Next, we define factories that create the quantum generator and a function that calculates the gradients for the quantum generator.

In [6]:
def create_generator(qnn: QuantumCircuit,
                     parameters: Union[ParameterVector, ParameterExpression, List[ParameterExpression]]) -> TorchConnector:
    """
    Factory to create the quantum generator from a given parameterized quantum circuit.
    Args:
        qnn: Quantum neural network ansatz given as a quantum circuit.
        parameters: The parameters of the quantum neural network which are trained.
    Returns:
        Quantum neural network compatible with PyTorch
    """
    circuit_qnn = CircuitQNN(qnn, input_params=[], weight_params = parameters,
                             quantum_instance=qi, sampling=True, sparse=False,
                             input_gradients=True, interpret=lambda x: grid_elements[x])
    # We use the Qiskit TorchConnector to ensure compatibility with PyTorch
    return TorchConnector(circuit_qnn)

def generator_grad(qnn: QuantumCircuit,
                   parameters: Union[ParameterVector, ParameterExpression, List[ParameterExpression]],
                   param_values: Iterable,
                   grad_method: str = 'param_shift') -> Iterable:
    """
    The function returns the gradient values for the quantum generator given the
    underlying parameterized quantum circuit.
    Args:
        qnn: Quantum neural network ansatz given as a quantum circuit.
        parameters: The parameters of the quantum neural network which are trained.
        param_values: The current values of the quantum neural network parameters.
        grad_method: Method used to compute the gradients {'param_shift', 'lin_comb', 'fin_diff'}
    Returns:
        List of gradients for the sampling probabilities of the quantum neural network.
    """
    grad = Gradient(grad_method=grad_method).gradient_wrapper(StateFn(qnn), parameters, backend=qi)
    grad_values = grad(param_values)
    return grad_values.tolist()

### Definition of classical discriminator
Next, we define factories that create the classical discriminators. The underlying gradients can be automatically computed with PyTorch.

In [7]:
class create_discriminator(nn.Module):
    """
    Factory to create the quantum generator from a given parameterized quantum circuit.
    """
    def __init__(self):
        super(create_discriminator, self).__init__()

        self.Linear_in = nn.Linear(len(data_dim), 20)
        self.Leaky_ReLU = nn.LeakyReLU(0.2, inplace=True)
        # self.Linear50 = nn.Linear(50, 20)
        self.Linear20 = nn.Linear(20, 1)
        self.Sigmoid = nn.Sigmoid()

    def forward(self, input: Tensor) -> Tensor:
        x = self.Linear_in(input)
        x = self.Leaky_ReLU(x)
        # x = self.Linear50(x)
        # x = self.Leaky_ReLU(x)
        x = self.Linear20(x)
        x = self.Sigmoid(x)
        return x

### Definition of the quantum neural network ansatz
Now, we are ready to define the parameterized quantum circuit $G\left(\boldsymbol{\theta}\right)$ with $\boldsymbol{\theta} = {\theta_1, ..., \theta_k}$ which corresponds to our quantum generator.


In [8]:
# sum(data_dim) corresponds to the total number of qubits in our quantum circuit (qc)
qc = QuantumCircuit(sum(data_dim))
qc.h(qc.qubits)
# We choose a hardware efficient ansatz.
twolocal = TwoLocal(sum(data_dim), "ry", "cx", reps=2, entanglement="circular")
qc.compose(twolocal, inplace=True)

### Definition of the loss functions
We want to train the generator and the discriminator with binary cross entropy as loss function:
$$L\left(\boldsymbol{\theta}\right)=\sum_jp_j\left(\boldsymbol{\theta}\right)\left[y_j\log(x_j) + (1-y_j)\log(1-x_j)\right],$$
where $x_j$ refers to a data sample and $y_j$ to the corresponding label.

In [9]:
# Generator loss function
g_loss_fun = nn.BCELoss()
# Discriminator loss function
d_loss_fun = nn.BCELoss()

### Evaluation of custom gradients for the generator BCE loss function
The evaluation of custom gradients for the quantum generator requires us to combine quantum gradients $\frac{\partial p_j\left(\boldsymbol{\theta}\right)}{\partial \theta_l}$ that we compute with Qiskit's gradient framework with the binary cross entropy as follows:
$$\frac{\partial L\left(\boldsymbol{\theta}\right)}{\partial \theta_l} = \sum_j \frac{\partial p_j\left(\boldsymbol{\theta}\right)}{\partial \theta_l} \left[y_j\log(x_j) + (1-y_j)\log(1-x_j)\right].$$

In [10]:
def g_loss_fun_grad(qnn: QuantumCircuit,
                    parameters: Union[ParameterVector, ParameterExpression, List[ParameterExpression]],
                    param_values: Iterable,
                    discriminator_: nn.Module,
                    grad_method: str = 'param_shift') -> Iterable:
    """
    Custom gradient of the generator loss function considering the custom gradients of the quantum generator
    Args:
        qnn: Quantum neural network ansatz given as a quantum circuit.
        parameters: The parameters of the quantum neural network which are trained.
        param_values: The current values of the quantum neural network parameters.
        discriminator_: Classical neural network representing the discriminator.
        grad_method: Method used to compute the gradients {'param_shift', 'lin_comb', 'fin_diff'}
    Returns:
        List of gradient values, i.e., the gradients of the loss function w.r.t. the quantum neural network parameters
    """
    grads = generator_grad(qnn, parameters, param_values, grad_method = grad_method)
    loss_grad = ()
    for j, grad in enumerate(grads):
        cx = grad[0].tocoo()
        input = []
        target = []
        weight = []
        for index, prob_grad in zip(cx.col, cx.data):
            input.append(grid_elements[index])
            target.append([1.])
            weight.append([prob_grad])
        bce_loss_grad = F.binary_cross_entropy(discriminator_(Tensor(input)), Tensor(target), weight=Tensor(weight))
        loss_grad += (bce_loss_grad, )
    loss_grad = stack(loss_grad)
    return loss_grad

### Relative entropy as benchmarking metric
The relative entropy describes a distance metric for distributions. Hence, we can use it to benchmark how close/far away the trained distribution is from the target distribution.

In [11]:
def get_rel_entr(gen_data) -> float:
    """Get relative entropy between target and trained distribution"""
    prob_gen = np.zeros(len(grid_elements))
    for j, item in enumerate(grid_elements):
        for gen_item in gen_data.detach().numpy():
            if np.allclose(np.round(gen_item, 6), np.round(item, 6), rtol=1e-5):
                prob_gen[j] += 1
    prob_gen=prob_gen/len(gen_data)
    prob_gen = [1e-8 if x == 0 else x for x in prob_gen]
    rel_entr = entropy(prob_gen, prob_data)
    return rel_entr

### Definition of the optimizers
In order to train the generator and discriminator, we need to define optimization schemes. In the following, we employ a momentum based optimizer called Adam [1].

**References:**

[1] Kingma et al.,
    `Adam: A method for stochastic optimization <https://arxiv.org/abs/1412.6980>`_


In [12]:
# Initialize generator and discriminator
generator = create_generator(qc, twolocal.parameters)
discriminator = create_discriminator()

lr=0.01 #learning rate
b1=0.9 #first momentum parameter
b2=0.999 #second momentum parameter
n_epochs=1 #number of training epochs

#optimizer for the generator
optimizer_G = Adam(generator.parameters(), lr=lr, betas=(b1, b2))
#optimizer for the discriminator
optimizer_D = Adam(discriminator.parameters(), lr=lr, betas=(b1, b2))

### Training
Now, we are ready to train our model.

In [13]:
# Relative entropy list
rel_entr_list = []
# Generator loss list
g_loss_list = []
# Discriminator loss list
d_loss_list = []

for epoch in range(n_epochs):
    rel_entr = []
    g_loss = []
    d_loss = []
    for i, data in enumerate(dataloader):
        # Adversarial ground truths
        valid = Variable(Tensor(data.size(0), 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(data.size(0), 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_data = Variable(data.type(Tensor))
        # Generate a batch of images
        gen_data = generator()

        # Evaluate Relative Entropy
        rel_entr.append(get_rel_entr(gen_data))

        # ---------------------
        #  Train Discriminator
        # ---------------------
        optimizer_D.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        disc_data = discriminator(real_data)
        real_loss = d_loss_fun(disc_data, valid)
        fake_loss = d_loss_fun(discriminator(gen_data), fake)  # (discriminator(gen_data).detach(), fake)
        d_loss = (real_loss + fake_loss) / 2

        # for name, param in discriminator.state_dict().items():
        #     print(name, param.detach())

        # print('d loss ', d_loss)

        d_loss.backward(retain_graph=True)
        optimizer_D.step()

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # # Loss measures generator's ability to fool the discriminator
        g_loss = g_loss_fun(discriminator(gen_data), valid)
        g_loss.retain_grad = True
        g_loss_grad = g_loss_fun_grad(qc, qc.ordered_parameters, generator.weight.data.numpy(), discriminator)

        g_loss.backward(retain_graph=True)
        for j, param in enumerate(generator.parameters()):
            param.grad = g_loss_grad
        optimizer_G.step()

        g_loss.append(g_loss.item())
        d_loss.append(d_loss.item())

        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
            % (epoch, n_epochs, i, len(dataloader), d_loss.item(), g_loss.item())
        )

    rel_entr_list.append(np.mean(rel_entr))
    g_loss_list.append(np.mean(g_loss))
    d_loss_list.append(np.mean(d_loss))

AttributeError: 'QuantumCircuit' object has no attribute 'ordered_parameters'

### Plots
Let's visualize what happened during the training.

In [ ]:
# Relative Entropy
plt.figure()
plt.title('Relative entropy')
plt.plot(rel_entr_list)
plt.xlabel('Iteration')
plt.ylabel('Relative entropy')
plt.show()

In [ ]:
# Loss
plt.figure()
plt.title('Loss')
plt.plot(g_loss_list, label = 'generator loss')
plt.plot(d_loss_list, label = 'discriminator loss')
plt.legend(loc='best')
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.show()
